In [112]:
import pandas as pd
import numpy as np

In [154]:
data_location = "Data/properatti.csv"
data = pd.read_csv(data_location)

In [156]:
#Agregue "place_name", y "place_with_paren_names" al DF para poder trabajarlas en conjunto con "state_name"
df_nuevo1 = data.loc[:,['property_type', 'country_name',"place_name", "place_with_parent_names", 'state_name', 'price', 'currency', 'price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2','price_per_m2', 'rooms','description', 'title']]
df_nuevo1.head()

,property_type,country_name,place_name,place_with_parent_names,state_name,price,currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_per_m2,rooms,description,title
0,PH,Argentina,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,62000.0,USD,62000.0,55.0,40.0,1550.000000,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,Argentina,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Bs.As. G.B.A. Zona Sur,150000.0,USD,150000.0,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Argentina,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,72000.0,USD,72000.0,55.0,55.0,1309.090909,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Argentina,Liniers,|Argentina|Capital Federal|Liniers|,Capital Federal,95000.0,USD,95000.0,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Argentina,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Buenos Aires Costa Atlántica,64000.0,USD,64000.0,35.0,35.0,1828.571429,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [ ]:
#place_name no se puede utilizar como columna de Localidad ya que hay datos mal cargados, por ejemplo ciertas filas con nombres de Barrios Privados. 
#Es por eso que trabaje la columna "place_with_parent_names", la splitie y obtuve la localidad de la misma. 

In [157]:
#Hize un regex para sacar los |Argentina del comienzo de "place_with_parent_names"
import re
patron_argentina = "\|Argentina\|"
patron_regex_arg = re.compile(patron_argentina)
df_nuevo1["place_with_parent_names"] = df_nuevo1["place_with_parent_names"].apply(lambda x: x if x is np.NaN else patron_regex_arg.sub("",x))
df_nuevo1.loc[:,["place_name", "place_with_parent_names", "state_name"]].head()

,place_name,place_with_parent_names,state_name
0,Mataderos,Capital Federal|Mataderos|,Capital Federal
1,La Plata,Bs.As. G.B.A. Zona Sur|La Plata|,Bs.As. G.B.A. Zona Sur
2,Mataderos,Capital Federal|Mataderos|,Capital Federal
3,Liniers,Capital Federal|Liniers|,Capital Federal
4,Centro,Buenos Aires Costa Atlántica|Mar del Plata|Cen...,Buenos Aires Costa Atlántica


In [158]:
#Splitie "place_with_parent_names" en 3 columnas distintas (provincia, localidad, info extra) y las agregue al DF.  
df_nuevo1[["Provincia","Localidad", "Info_extra"]] = df_nuevo1["place_with_parent_names"].str.split("|",n=2, expand=True)
df_nuevo1.loc[:,["Provincia","Localidad", "Info_extra"]]

,Provincia,Localidad,Info_extra
0,Capital Federal,Mataderos,
1,Bs.As. G.B.A. Zona Sur,La Plata,
2,Capital Federal,Mataderos,
3,Capital Federal,Liniers,
4,Buenos Aires Costa Atlántica,Mar del Plata,Centro|
...,...,...,...
121215,Capital Federal,Belgrano,
121216,Bs.As. G.B.A. Zona Norte,San Isidro,Beccar|
121217,Capital Federal,Villa Urquiza,
121218,Buenos Aires Costa Atlántica,Mar del Plata,Plaza Colón|


In [117]:
#Cree un nuevo DF con las 3 columnas del df original y con las columnas spliteadas para comparar la info de todas las columnas.
chequeo_data_frame = df_nuevo1.loc[:,["state_name", "place_name", "place_with_parent_names", "Provincia", "Localidad", "Info_extra"]]
chequeo_data_frame.head()

,state_name,place_name,place_with_parent_names,Provincia,Localidad,Info_extra
0,Capital Federal,Mataderos,Capital Federal|Mataderos|,Capital Federal,Mataderos,
1,Bs.As. G.B.A. Zona Sur,La Plata,Bs.As. G.B.A. Zona Sur|La Plata|,Bs.As. G.B.A. Zona Sur,La Plata,
2,Capital Federal,Mataderos,Capital Federal|Mataderos|,Capital Federal,Mataderos,
3,Capital Federal,Liniers,Capital Federal|Liniers|,Capital Federal,Liniers,
4,Buenos Aires Costa Atlántica,Centro,Buenos Aires Costa Atlántica|Mar del Plata|Cen...,Buenos Aires Costa Atlántica,Mar del Plata,Centro|


In [118]:
#Creo una mascara para aquellas propiedades sin Localidad(columna Localidad en blanco) para ver si puedo completar la localidad con la info de las otras columnas.
mask_propiedades_sin_localidad = df_nuevo1["Localidad"] == ""
#Para corroborar cree un DF con aquellas propiedades sin localidad y comparo columnas. 
prop_sin_localidad = chequeo_data_frame.loc[mask_propiedades_sin_localidad,:]
prop_sin_localidad

,state_name,place_name,place_with_parent_names,Provincia,Localidad,Info_extra
108,Capital Federal,Capital Federal,Capital Federal|,Capital Federal,,None
110,Capital Federal,Capital Federal,Capital Federal|,Capital Federal,,None
115,Capital Federal,Capital Federal,Capital Federal|,Capital Federal,,None
119,Capital Federal,Capital Federal,Capital Federal|,Capital Federal,,None
121,Capital Federal,Capital Federal,Capital Federal|,Capital Federal,,None
...,...,...,...,...,...,...
121203,Córdoba,Córdoba,Córdoba|,Córdoba,,None
121204,Córdoba,Córdoba,Córdoba|,Córdoba,,None
121205,Córdoba,Córdoba,Córdoba|,Córdoba,,None
121206,Córdoba,Córdoba,Córdoba|,Córdoba,,None


In [159]:
#Con estas igualdades chequeo que las columnas tienen la misma info y que no puedo obtener info adicional para completar la localidad de ciertas propiedas.
#son un total de 4780 propiedades sin localidad.
print((prop_sin_localidad["place_name"] == prop_sin_localidad["Provincia"]).all())
print((prop_sin_localidad["state_name"] == prop_sin_localidad["Provincia"]).all())

True
True


In [160]:
#chequeo si hay propiedades que al haber spliteado "place_with_parent_names" hayan quedado con misma Provincia y Localidad.
print((df_nuevo1["Provincia"] != df_nuevo1["Localidad"]).all())
mask_misma_prov_y_localidad = df_nuevo1["Provincia"] == df_nuevo1["Localidad"]
df_nuevo1.loc[mask_misma_prov_y_localidad, ["state_name", "place_name", "place_with_parent_names", "Provincia", "Localidad", "Info_extra"]]
#hay 7853 otras propiedades que tienen misma Provincia y Localidad, es decir que no cuentan con Localidad. 
#Total de 12633 propiedades que no cuentan con Localidad, unicamente provincia. Total del 10.42% de las propiedades.

False


,state_name,place_name,place_with_parent_names,Provincia,Localidad,Info_extra
10,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
32,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
87,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
88,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
89,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
...,...,...,...,...,...,...
120690,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
120966,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
120971,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
120979,Córdoba,Córdoba,Córdoba|Córdoba|,Córdoba,Córdoba,
